# Problem 1 - Extending the Lady Tasting Tea Experiment

The aim of this notebook, is to extend the classical *Lady Tasting Tea* experiment, as demonstrated in the module, to use a larger number of cups.

To achieve this, I will:

- Set up the original 8-cup experiment
- Set up extended 12-cup experiment,
- Use Numpy, to simulate both the the original 8 cup and extended 12-cup experiment
- Interpret the findings of the analysis

---

# Importing Required Libraries

In [1]:
# Mathematical functions from the standard library.
# https://docs.python.org/3/library/math.html
import math

# Permutations and combinations.
# https://docs.python.org/3/library/itertools.html
import itertools

# Random selections.
# https://docs.python.org/3/library/random.html
import random

# Numerical structures and operations.
# https://numpy.org/doc/stable/reference/index.html#reference
import numpy as np

# Plotting.
# https://matplotlib.org/stable/contents.html
import matplotlib.pyplot as plt

# Data manipulation and analysis.
# https://pandas.pydata.org/docs/
import pandas as pd

# Binomial coefficients (comb) from the math module.
# https://docs.python.org/3/library/math.html#math.comb
from math import comb


from scipy.stats import binom

---

# Experiment Set Up - Original and Extended

## Setup for Both Experiments

The designs of both the Experiments are detailed below:

1. **Original experiment as performed by Fisher:**
   - 8 cups in total
   - 4 cups with milk poured first
   - 4 cups with tea poured first

2. **Extended experiment  as carried out in this Notebook:**
   - 12 cups in total
   - 4 cups with milk poured first
   - 8 cups with tea poured first

For both experiments, the particiant is told there are exactly 4 milk-first cups and, in trn,
must try identify which 4 cups these are.

Under the null hypothesis (they possess no special ability to distuinguish between Milk First or Tea First), the participant is in essence
choosing 4 random cups at random from the total. The number of possible
guesses in each case is given by the binomial coefficient:

$$
\frac{n!}{k!(n-k)!} = \binom{n}{k}
$$

The expression above, represents the number of ways to choose $k$ cups from $n$ cups.


In the next two sections, I will set up both experiments and compute these potential combinations and probabilities of successfully identifying the four Milk First cups at random.


## Original Experiment - Cups of Tea

In [2]:
# --- Original 8-cup setup ---

original_num_cups = 8 # Setting the number of cups in the original experiment
original_num_milk_first = 4 # Setting the number of milk-first cups in the original experiment

original_num_comb = math.comb(original_num_cups, original_num_milk_first) # Calculating the number of combinations from the original experiment of choosing 4 cups from 8

# Exact probability of getting all 4 correct by chance.
original_prob_all_correct = 1 / original_num_comb # Calculating the exact probability of getting all 4 correct by chance in the original experiment

original_num_comb, original_prob_all_correct # Displaying the number of combinations and the exact probability of getting all 4 correct by chance in the original experiment

(70, 0.014285714285714285)

## Updated Experiment - Cups of Tea

In [3]:
# --- Extended 12-cup setup ---

ext_num_cups = 12 # Setting the number of cups in the extended experiment
ext_num_milk_first = 4 # Setting the number of milk-first cups in the extended experiment

ext_num_comb = math.comb(ext_num_cups, ext_num_milk_first) # Calculating the number of combinations from the extended setup of choosing 4 cups from 12

extended_prob_all_correct = 1 / ext_num_comb # Calculating the exact probability of getting all 4 correct by chance in the extended experiment

ext_num_comb, extended_prob_all_correct # Displaying the number of combinations and the exact probability of getting all 4 correct by chance in the extended experiment


(495, 0.00202020202020202)

---

# Running the Extended Experiment

## Identifying the number of Simulations to run

In [4]:
# Step 1: Define the domain / model
num_of_iters = [1_000, 5_000, 10_000, 50_000, 100_000, 1_000_000, 2_000_000] # Creating a list of different numbers of simulations to run

results = []  # Creating an empty list to store the results

# Step 2: Generate random inputs (simulate)
for n in num_of_iters:  # For Loop stating to run the below steps for each number of simulations in num_of_iters, where n is the number of simulations being run
    # We simulate n independent Bernoulli trials with success probability p (extended_prob_all_correct).
    # np.random.binomial(n=1, p=p, size=n) generates n values of either 1 or 0.
    # n=1 indicates each trial is a single Bernoulli trial.
    # p=extended_prob_all_correct is the probability of success on each trial.
    # size=n specifies that we want to generate n such trials i.e the number of simulations.
    # A value of 1 represents a success (all 4 answers correct by chance), and a value of 0 represents a failure.
    
    trials = np.random.binomial(n=1, p=extended_prob_all_correct, size=n)

# Step 3: Deterministic computation on each input

    # Taking the mean of trials gives the estimated probability of success based on these simulated Bernoulli trials.
    # A value of 1 is counted as a success and 0 as a failure, so the mean gives the proportion of successes.
    estimate = trials.mean()

# Step 4: Aggregate / summarize the results
    # Calculating the difference between the estimate and the true probability
    diff = estimate - extended_prob_all_correct  

    # Calculating variance of the estimate, using p(1 - p) / n where p is the true probability..
    variance = extended_prob_all_correct * (1 - extended_prob_all_correct) / n

    # Calculating the square root of the variance to get the standard error.
    standard_error = np.sqrt(variance)

    # Validating that the standard error calculation using numpy matches our manual calculation.
    np_standard_error = np.sqrt(extended_prob_all_correct * (1 - extended_prob_all_correct)) / np.sqrt(n)

    # Appending the number of simulations, estimate, and difference to the results list
    results.append([n, round(estimate,6), round(diff, 6), round(variance, 6), round(standard_error, 6), round(np_standard_error, 6)])

# Creating a DataFrame from the results list with specified column names
df_results = pd.DataFrame(
    results,
    columns=["Simulations", "Estimate", "Difference from Exact Probability", "Variance", "Standard Error", "Numpy Standard Error"]
)

df_results  # Displaying the results DataFrame

,Simulations,Estimate,Difference from Exact Probability,Variance,Standard Error,Numpy Standard Error
0,1000,0.000000,-0.002020,0.000002,0.001420,0.001420
1,5000,0.001600,-0.000420,0.000000,0.000635,0.000635
2,10000,0.001900,-0.000120,0.000000,0.000449,0.000449
3,50000,0.002400,0.000380,0.000000,0.000201,0.000201
4,100000,0.002130,0.000110,0.000000,0.000142,0.000142
5,1000000,0.001977,-0.000043,0.000000,0.000045,0.000045
6,2000000,0.002031,0.000011,0.000000,0.000032,0.000032


In [5]:
standard_error = np.sqrt(extended_prob_all_correct * (1 - extended_prob_all_correct)) / np.sqrt(n)
standard_error

3.1749966960609514e-05

### References to Materials for this Code Block

1. Steps in the Monte Carlo Simulation - https://bookdown.org/manuele_leonelli/SimBook/steps-of-monte-carlo-simulation.html
2. Creating an Empty List - https://www.geeksforgeeks.org/python/declare-an-empty-list-in-python/
3. Standard Error vs Standard Deviation - https://www.investopedia.com/ask/answers/042415/what-difference-between-standard-error-means-and-standard-deviation.asp
4. Standard Error vs Standard Deviation - https://statisticsbyjim.com/basics/difference-standard-deviation-vs-standard-error/
5. For Loops - https://www.geeksforgeeks.org/python/loops-in-python/
6. Numpy Random.binomial - https://www.geeksforgeeks.org/numpy/binomial-distribution-in-numpy/
7. Numpy Random.binomial - https://numpy.org/doc/stable/reference/random/generated/numpy.random.binomial.html
8. Running Bernoulli trials - https://medium.com/@heyamit10/understanding-bernoulli-distribution-in-numpy-308736f0f963
9. Monte Carlo Simulation and Bernoulli Distribution in Python - https://medium.com/@polanitzer/monte-carlo-simulation-and-bernoulli-distribution-in-python-predict-next-years-tax-exemption-ec6f3d8b6ba9
10. Monte Carlo Simulation of Bernoulli Trials - https://www.geeksforgeeks.org/r-language/monte-carlo-simulation-of-bernoulli-trials-in-r/
11. Bernoulli Distribution - https://www.datacamp.com/tutorial/bernoulli-distribution
12. Error Estimation - https://www.oceanopticsbook.info/view/monte-carlo-simulation/level-2/error-estimation
13. Bernoulli Error - https://www.datacamp.com/tutorial/bernoulli-distribution
14. Monte Carlo Error Calculation - https://rpubs.com/Mabrouk12/Monte-Carlo-Error
15. Monte Carlo - Standard Error - https://www.mathworks.com/help/econ/monte-carlo-simulation-of-conditional-variance-models.html
16. Calculating the Variance - https://statproofbook.github.io/P/bern-var.html
17. Calculating Variance - https://stats.libretexts.org/Bookshelves/Probability_Theory/Probability_Mathematical_Statistics_and_Stochastic_Processes_%28Siegrist%29/08%3A_Set_Estimation/8.03%3A_Estimation_in_the_Bernoulli_Model
18. Calculating Square Error - https://docs.statsig.com/experiments/statistical-methods/variance
19. Standard Error - https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Introductory_Statistics_%28Hannah_Seidler-Wright%29/06%3A_Inference_Involving_a_Single_Population_Proportion/6.01%3A_The_Sampling_Distribution_of_Sample_Proportions
20. Panda Dataframes - https://www.w3schools.com/python/pandas/pandas_dataframes.asp
21. .Append - https://www.geeksforgeeks.org/python/python-list-append-method/
22. Standard Error Interpretation - https://statisticsbyjim.com/hypothesis-testing/standard-error-mean/
23. Standard Error Interpretation - https://www.datacamp.com/tutorial/what-is-standard-error
24. Standard Error Interpretation - https://www.investopedia.com/terms/s/standard-error.asp

## Using NumPy to run the Extended Experiment with 12 Cups (4 Milk First)

Again, under the null hypothesis, the participant is identifying the 4 Milk First cups at random from an extended number of 12 cups total.  

In order to estimate the probability of all 4 milk-first cups being correctly identified in the extended 12-cup experiment, we will run the experiment 100,000 times using NumPy. 

The simulation follows the Monte Carlo method described above, where each simulated run corresponds to an independent Bernoulli trial. 
The choice of 100,000 simulations is based on practical Monte Carlo considerations: using a large number of repetitions reduces sampling variability and ensures our estimate is close to the exact probability.

The exact probability is $1/495$.  


### Running Extended Experiment

In [6]:
ext_milk_first_labels = set(range(ext_num_milk_first)) # Defining the Milk-First cup labels in the extended experiment

num_sims = 100000 # Setting the number of simulations to run based on Monte Carlo Simulation.

rng = np.random.default_rng() # Using NumPy's default random number generator to allow for the simulation to be repeatable

ext_num_of_successes = 0 # Setting the initial number of successes to zero

for i in range(num_sims): # Stating to run the below steps for number of simulations in num_sims
    # Randomly guessing which cups are milk-first without replacement.
    # Assigning the guessed milk-first cup labels to guessed_labels.
    # Using rng.choice to randomly select ext_num_milk_first unique cup labels from the total ext_num_cups.
    # ext_num_cups: Total number of cups (12 in this case).
    # size=ext_num_milk_first: Number of milk-first cups to guess (4 in this case).
    # replace=False: Ensures that the same cup label is not chosen more than once (no replacement).
    ext_guessed_labels = rng.choice(ext_num_cups, 
                                size=ext_num_milk_first,
                                replace=False)
    
    if set(ext_guessed_labels) == ext_milk_first_labels: # Checking if the guessed labels match the actual milk-first labels
        ext_num_of_successes += 1 # Incrementing the number of successes if the guess is correct

estimated_prob_all_correct_extended = ext_num_of_successes / num_sims # Calculating the estimated probability of getting all 4 correct by chance in the extended experiment based on the simulation

ext_num_of_successes, num_sims, estimated_prob_all_correct_extended # Displaying the number of successes, number of simulations, and the estimated probability from the simulation


(187, 100000, 0.00187)

### Running Simulation of Original Experiment

In [7]:
orig_milk_first_labels = set(range(original_num_milk_first)) # Defining the Milk-First cup labels in the original experiment


orig_num_of_successes = 0 # Setting the initial number of successes to zero

for i in range(num_sims): # Stating to run the below steps for number of simulations in num_sims
    # Randomly guessing which cups are milk-first without replacement.
    # Assigning the guessed milk-first cup labels to guessed_labels.
    # Using rng.choice to randomly select original_num_milk_first unique cup labels from the total original_num_cups.
    # original_num_cups: Total number of cups (8 in this case).
    # size=original_num_milk_first: Number of milk-first cups to guess (4 in this case).
    # replace=False: Ensures that the same cup label is not chosen more than once (no replacement).
    orig_guessed_labels = rng.choice(original_num_cups, 
                                size=original_num_milk_first,
                                replace=False)
    
    if set(orig_guessed_labels) == orig_milk_first_labels: # Checking if the guessed labels match the actual milk-first labels
        orig_num_of_successes += 1 # Incrementing the number of successes if the guess is correct

estimated_prob_all_correct_orig = orig_num_of_successes / num_sims # Calculating the estimated probability of getting all 4 correct by chance in the extended experiment based on the simulation

orig_num_of_successes, num_sims, estimated_prob_all_correct_orig # Displaying the number of successes, number of simulations, and the estimated probability from the simulation


(1471, 100000, 0.01471)

### References to Materials for this Code Block
1. Sets vs Lists - https://www.geeksforgeeks.org/python/sets-vs-lists-python/
2. Sets in Python - https://www.geeksforgeeks.org/python/sets-in-python/
3. Ranges in Python - https://mimo.org/glossary/python/range-function
4. Set & Ranges - https://medium.com/@sjalexandre/python-tutorial-ranges-sets-tuples-a2f5bd0564fd
5. Set & Ranges - https://realpython.com/python-sets/
6. Success Counter - https://www.geeksforgeeks.org/python/python-program-for-biased-coin-flipping-simulation/
7. Success Counter - https://www.khanacademy.org/python-program/coin_flip_simulation/6383274622369792
8. Troubleshooting issue with For Loop (Without using Range) - https://realpython.com/python-for-loop/
9. For loop with Range - https://www.geeksforgeeks.org/python/python-loop-through-a-range/
10. For Loop with Range - https://www.w3schools.com/python/gloss_python_for_range.asp
11. Numpy Random - https://realpython.com/numpy-random-number-generator/
12. Numpy Random - https://numpy.org/doc/stable/reference/random/index.html
13. Numpy Random - https://medium.com/@prathik.codes/understanding-numpy-random-number-generator-rng-class-and-its-machine-learning-applications-c2020005e274
14. Numpy Random Seed - https://numpy.org/doc/stable/reference/random/generator.html
15. Random.Choice - https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.choice.html
16. Random.Choice - https://www.geeksforgeeks.org/python/random-choices-method-in-python/
17. Understanding Numpy Random Default RNG - https://blog.scientific-python.org/numpy/numpy-rng/
18. Understanding Numpy Random Default RNG  - https://proclusacademy.com/blog/nupmy-generate-random-numbers-distributions/
19. Checking if the guess is right - https://medium.com/@harshitbasti/making-a-guess-game-in-python-b06e0db29c67
20. Checking if guesses is right - https://realpython.com/courses/python-is-identity-vs-equality/
21. Checking if guess is right - https://www.geeksforgeeks.org/python/number-guessing-game-in-python/
22. Counting Correct Guesses - https://realpython.com/python-counter/
23. Calculating Probabilities - https://www.geeksforgeeks.org/maths/coin-toss-probability-formula/
24. Calculating Probabilities - https://www.datacamp.com/tutorial/statistics-python-tutorial-probability-1


---

# Interpretation of Results

In [8]:


df_prob = pd.DataFrame({
    "Experiment": ["Original (8 cups)", "Extended (12 cups)"],
    "Total Cups": [8, 12],
    "Milk-First Cups": [4, 4],
    "Exact Probability": [original_prob_all_correct, extended_prob_all_correct],
    "Simulated Probability": [estimated_prob_all_correct_orig, estimated_prob_all_correct_extended],
})

df_prob


,Experiment,Total Cups,Milk-First Cups,Exact Probability,Simulated Probability
0,Original (8 cups),8,4,0.014286,0.01471
1,Extended (12 cups),12,4,0.002020,0.00187


## Interpreting the Estimated Probabilities

The table above shows both the exact and simulated probabilities of correctly identifying all milk-first cups purely by random guessing,that is, achieving a perfect score in each experiment.

### 2. Interpretation of the Results

- **Original 8-cup experiment:**  
  The exact probability of a perfect score is roughly 1/70 or 0.0143 for the Original 8 Cup Experiment.  
  This means that even with no genuine ability, a participant would, on average, guess all 4 milk-first cups correctly about 1.4% of the time.

- **Extended 12-cup experiment:**  
  The exact probability decreases to 1/495 or 0.0020 for the Extended 12 Cup Experiment 
  This means that even with no genuine ability, a participant would, on average, guess all 4 milk-first cups correctly about 0.2% of the time when the number of cups is increased to 12.

By increasing the total number of cups from 8 to 12, while keeping the number of milk-first cups fixed at 4, it makes it much harder for a random guesser to pick the correct combination. As a result, the probability of achieving a perfect score by chance becomes much smaller.

In fact, the liklihood of achieving a perfect score in the 12-cup version is over 7 times **less likely** than in the original experiment, under the assumption of Random Guessing.

---

### 3. Interpreting the p-values

Under the null hypothesis (they possess no special ability to distuinguish between Milk First or Tea First), the participant is in essence
choosing 4 cups at random from the all cups. A perfect score in this situation is therefore extremely unlikely, and the p-value tells us *how unlikely* it is.

In the scenario that we use a perfect score, as the criteria to detect ability:

- **Original 8-cup experiment:**  
  - The chance of a perfect guess is about 0.014 (1.4%).  
  - This is below the 5%/ 0.05 threshold level low enough to denote it as statistically significant.
    - This is turn, may indicate an innate ability to actually identify the cups that were Milk First.  
  - However, if we were to use the stricter 1%/ 0.01 threshold level, this would not be deemed to be statistically significant.

- **Extended 12-cup experiment:**  
  - The chance of a perfect guess is about 0.002 (0.2%), when the experiment is extended to have 12 cups.   
  - This is below both the 5% and 1% levels, meaning a perfect score provides very strong reasoning to believe a participant has an ability to tell the difference.

- **Summary of Analysis:** 
  - Based on the above findings, we would reject the null hypothesis in both experiments, but the extended version provides much stronger evidence, as the original experiment only meets the 5% significance level, while the extended experiment is also significant at the stricter 1% level.

  - As the extended experiment results in an extremely small p-value (≈ 0.002), it would still be considered significant even under the more stringent thresholds proposed by Benjamin & Berger (2019), who argued for lowering the traditional 0.05 standard. In the case of a stricter threshold, the original experiment, by contrast, would not meet it.

  - However, in standard practice, researchers typically retain the conventional 5% or 1% significance levels, in which case both experiments allow rejection of the null hypothesis, with the extended design providing much stronger evidence.


---



# References Used for Research/ Knowledge Building around Problem


1. Binomial Distribution - https://www.geeksforgeeks.org/python/python-binomial-distribution/
2.  Binomial Distribution - https://campus.datacamp.com/courses/introduction-to-statistics-in-python/random-numbers-and-probability-2?ex=13
3.  Binomial Distribution - https://docs.scipy.org/doc//scipy-1.9.2/reference/generated/scipy.stats.binom.html
4.  Binomial Distribution - https://www.analyticsvidhya.com/blog/2022/05/everything-you-need-to-know-about-binomial-distribution/
5.  Binomial Distribution - https://towardsdatascience.com/binomial-distribution-ec76d74952c4/
6.  Bernouli Trials - https://www.geeksforgeeks.org/r-language/monte-carlo-simulation-of-bernoulli-trials-in-r/
7.  Bernouli Trials- https://www.geeksforgeeks.org/maths/bernoulli-trials-binomial-distribution/
8.  Bernouli Trials- https://statisticsbyjim.com/glossary/bernoulli-trial/
9.  Bernouli Trials- https://www.analyticsvidhya.com/blog/2024/11/bernoulli-distribution/
10. Bernouli Trials- https://github.com/uros-bojanic/bernoulli-trials
11. Bernouli Trials - https://codeinstitute.net/blog/bernoulli-distribution-its-uses/#:~:text=Using%20Bernoulli%20Distribution%20in%20Code&text=For%20example%2C%20in%20Python%2C%20the,trials%20and%20obtain%20binary%20outcomes.
12. Monte Carlo Simulation and Bernouli - https://medium.com/@polanitzer/monte-carlo-simulation-and-bernoulli-distribution-in-python-predict-next-years-tax-exemption-ec6f3d8b6ba9
13. Monte Carlo Simulation - https://medium.com/@miyoko_shimura/monte-carlo-simulation-using-python-random-sampling-by-calculating-%CF%80-be3cdd326361
14. Monte Carlo Simulation - https://hhundman.medium.com/monte-carlo-simulation-in-python-d3e21efbd7e6
15. Monte Carlo Simulation - https://medium.com/@polanitzer/binomial-distribution-in-python-estimate-the-probability-to-default-of-6-companies-from-their-bond-2a9dd08c6169
16. Random Number Generator - https://realpython.com/numpy-random-number-generator/
17. Markdown - https://www.geeksforgeeks.org/machine-learning/markdown-cell-in-jupyter-notebook/
18. Markdown - https://stackoverflow.com/questions/36288670/how-to-programmatically-generate-markdown-output-in-jupyter-notebooks
19. Markdown - https://www.datacamp.com/tutorial/markdown-in-jupyter-notebook
20. Markdown - https://www.markdownguide.org/basic-syntax/
21. p-Value Interpretation - https://statisticsbyjim.com/hypothesis-testing/interpreting-p-values/
22. p-Value Interpretation - https://www.geeksforgeeks.org/machine-learning/p-value/
23. Benjamin, D. J., & Berger, J. O. (2019). Three recommendations for improving the use of p-values. The American Statistician, 73(sup1), 186–191. https://doi.org/10.1080/00031305.2018.1543135